### Simulation  Experiments

In [2]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
from grid_world.grid_world import GridWorld
from DMEIRL.DeepMEIRL_FC import DMEIRL
from grid_world.trajGen_grid_world import GridWorld_trajGen
from utils import utils

from grid_world.data_parser import DataParser
from grid_world.grid_world import GridWorld


import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

no background image
no background image


In [3]:
parser = DataParser(width=10,height=10)
from PIL import Image
import os
from tqdm import tqdm
tracks_path = 'demo_label/train/'
file_names = os.listdir(tracks_path)
#read images
imgs = []
for file_name in file_names:
    imgs.append(Image.open(tracks_path + "/" + file_name))

#parse images
for i in tqdm(range(len(imgs))):
    parser.ParseEnvironmentFromImage(imgs[i],file_names[i].split('.')[0],save_path='demo_env&feature')

100%|██████████| 4/4 [00:00<00:00, 151.95it/s]


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import math

# 处理并显示 100_75 目录下的图像
tracks_path = r'wifi_track_data/dacang/grid_data/env_imgs/100_75'
file_names = os.listdir(tracks_path)

# 读取图像
imgs = []
for file_name in file_names:
    imgs.append(Image.open(tracks_path + "/" + file_name))

# 计算显示的行数和列数
cols = 3  # 每行显示的图像数
rows = math.ceil(len(imgs) / cols)

# 创建子图
fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
# 确保axes是一维数组，便于处理
axes = axes.flatten() if len(imgs) > 1 else [axes]

# 显示图像
for i, (img, file_name) in enumerate(zip(imgs, file_names)):
    axes[i].imshow(img)
    axes[i].set_title(f'Image {file_name}')
    axes[i].axis('off')

# 隐藏多余的子图
for i in range(len(imgs), len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# 解析图像
for i in tqdm(range(len(imgs))):
    parser.ParseEnvironmentFromImage(imgs[i], file_names[i].split('.')[0])


# Figure 5 

figure 5 (b): The labeling of artificial environmental elements has already been prepared and stored in a .npy file located within 'features_folderPath'.

figure 5 (C): Based on this formula, the reward map for the artificial environment can be derived using the following code.

In [4]:
n = np.nan
state_mat = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, n, n, n, n],
                       [ 0, 0, 0, n, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, 0, 0, 0, 0],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],
                       [ 0, 0, 0, n, 0, 0, 8, 8, 8, 8],])

world_trajGen = GridWorld_trajGen(10,10,states_matrix=state_mat,
                                  features_folderPath='demo_env&feature/features_grid/0312_10x10',
                                  rewards_mul=[1,-2,-1,-1],
                                  init_states=list(range(0,10)),
                                  trans_prob=0.6,discount=0.95)
world_trajGen.ShowRewards(title='reward map for the artificial environment')

# Table 1 

Generated expert trajectory data can be derived using the following code.

In [5]:
world_trajGen.GenerateTrajectories(traj_count=500,traj_length=20,save=True)

100%|██████████| 500/500 [00:00<00:00, 2236.36it/s]


,m,trajs
0,1,"[(5, 1, 15), (15, 1, 25), (25, 1, 35), (35, 1,..."
1,2,"[(1, 4, 2), (2, 4, 3), (3, 4, 3), (3, 4, 4), (..."
2,3,"[(1, 4, 0), (0, 4, 10), (10, 2, 10), (10, 2, 0..."
3,4,"[(0, 4, 0), (0, 4, 0), (0, 4, 1), (1, 4, 2), (..."
4,5,"[(2, 4, 3), (3, 4, 4), (4, 1, 14), (14, 1, 24)..."
...,...,...
495,496,"[(0, 4, 0), (0, 4, 1), (1, 4, 1), (1, 4, 1), (..."
496,497,"[(9, 1, 19), (19, 1, 18), (18, 1, 28), (28, 3,..."
497,498,"[(6, 1, 16), (16, 1, 26), (26, 3, 26), (26, 3,..."
498,499,"[(8, 3, 7), (7, 3, 7), (7, 3, 6), (6, 1, 6), (..."


# Figure 6

figure 6 (a): The total number of times each expert visited various states were counted by the `world_trajGen.ShowTrajs_Experts()`.

figure 6 (b): Logarithmic value of expert track count were driven by `world.ShowGridWorld_Count_log`.

In [6]:
world_trajGen.ShowTrajs_Experts()

In [7]:
world = GridWorld(
    environments_img_folderPath='demo_label/train/',
    expert_trajs=world_trajGen.df_trajs_experts,
    width=10,
    height=10,
    active_all=True,
    trans_prob=0.6,
    discount=0.95,
    real_reward_mat = world_trajGen.real_rewards_matrix
)
world.ShowGridWorld_Count_log(title='Logarithmic value of expert track count')

trajs all avg length: 20


parsing environments from folder:: 100%|██████████| 4/4 [00:00<00:00, 184.96it/s]


# Figure 8

figure 8 (a): the reward values reconstructed by the neural network were driven by the following code.The default number of iterations is 100 and the output function is tanh.

In [12]:
dme = DMEIRL(world,layers = (16,32,64,32,16),lr = 0.0001,log=utils.date+'_tanh',log_dir='test_run')
rewards_result2,best_iter2,rewards2 = dme.train(n_epochs=100,save = False,demo=True)

100%|██████████| 100/100 [02:19<00:00,  1.39s/it]


In [13]:
len(rewards_result2)

100

In [14]:
world.ShowRewardsResult(rewards2[-1],title='DMEIRL奖励复原结果')

In [15]:
world.ShowRewardsAnimation(rewards2,title='DMEIRL奖励复原动画')

In [9]:
dme.SaveModel('tanh_model.pth')